In [1]:
#Import libraries used to produce desired results
import pandas as pd
import numpy as np
from pprint import pprint
from datetime import datetime,date
from geopy.geocoders import Nominatim
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=False)

# initialize Nominatim API (geopy.geocoders)
geolocator = Nominatim(user_agent="geoapiExercises")

In [2]:
#prep indices for data-merge loop
indices = np.arange(1,13)

#initial DataFrame for merge data results
data_col2_df = pd.DataFrame()

#loop the constructions of the DataFrame from CSV sources downloaded from Citibike
for i in indices:
    data_col2_df = data_col2_df.append(pd.read_csv(f"Archive/2014-{i} - Citi Bike trip data.csv"))

#Clean gender results based on Citibike documentation
data_col2_df["gender"].replace({0:"Unknown",1:"Male",2:"Female"}, inplace=True)

#Clean useless "\N" results from DataFrame
data_col2_df["birth year"].replace(r"\N", np.nan, inplace=True)

#Convert birth year to number
data_col2_df["birth year"] = data_col2_df["birth year"].astype(np.float64)

#drop NaN results
data_col2_df = data_col2_df.dropna(axis=0, how="any")

#Convert dates as strings to datetime format for proper data anaylsis
data_col2_df['starttime'] = pd.to_datetime(data_col2_df['starttime'])
data_col2_df['stoptime'] = pd.to_datetime(data_col2_df['stoptime'])
data_col2_df['Date'] = pd.to_datetime(data_col2_df['starttime'], format="%Y-%b-%d")

#view dtypes output to esnure proper conversion
data_col2_df.dtypes


tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                        float64
gender                             object
Date                       datetime64[ns]
dtype: object

In [3]:
#view cleaned DataFrame
data_col2_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,Date
0,471,2014-01-01 00:00:06,2014-01-01 00:07:57,2009,Catherine St & Monroe St,40.711174,-73.996826,263,Elizabeth St & Hester St,40.717290,-73.996375,16379,Subscriber,1986.0,Male,2014-01-01 00:00:06
1,1494,2014-01-01 00:00:38,2014-01-01 00:25:32,536,1 Ave & E 30 St,40.741444,-73.975361,259,South St & Whitehall St,40.701221,-74.012342,15611,Subscriber,1963.0,Male,2014-01-01 00:00:38
2,464,2014-01-01 00:03:59,2014-01-01 00:11:43,228,E 48 St & 3 Ave,40.754601,-73.971879,2022,E 59 St & Sutton Pl,40.758491,-73.959206,16613,Subscriber,1991.0,Male,2014-01-01 00:03:59
3,373,2014-01-01 00:05:15,2014-01-01 00:11:28,519,Pershing Square N,40.751884,-73.977702,526,E 33 St & 5 Ave,40.747659,-73.984907,15938,Subscriber,1989.0,Male,2014-01-01 00:05:15
4,660,2014-01-01 00:05:18,2014-01-01 00:16:18,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,436,Hancock St & Bedford Ave,40.682166,-73.953990,19830,Subscriber,1990.0,Male,2014-01-01 00:05:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399064,553,2014-12-31 23:55:22,2015-01-01 00:04:35,82,St James Pl & Pearl St,40.711174,-74.000165,2008,Little West St & 1 Pl,40.705693,-74.016777,16908,Subscriber,1982.0,Male,2014-12-31 23:55:22
399065,1355,2014-12-31 23:56:37,2015-01-01 00:19:12,325,E 19 St & 3 Ave,40.736245,-73.984738,327,Vesey Pl & River Terrace,40.715338,-74.016584,20964,Subscriber,1971.0,Male,2014-12-31 23:56:37
399066,431,2014-12-31 23:57:03,2015-01-01 00:04:14,536,1 Ave & E 30 St,40.741444,-73.975361,491,E 24 St & Park Ave S,40.740964,-73.986022,20845,Subscriber,1963.0,Male,2014-12-31 23:57:03
399067,332,2014-12-31 23:57:25,2015-01-01 00:02:57,293,Lafayette St & E 8 St,40.730287,-73.990765,317,E 6 St & Avenue B,40.724537,-73.981854,15374,Subscriber,1984.0,Male,2014-12-31 23:57:25


In [4]:
#Construct separate station DateFrame for more responsive Tableau performance/anaylsis

#clone prior DataFrame retaining only the columns desired
stations_df = data_col2_df[['start station id','start station name','start station latitude','start station longitude']]
#append end station data for completeness
stations_df.append(data_col2_df[['end station id','end station name','end station latitude','end station longitude']])
#drop duplicates by 'station id'
stations_df = stations_df.drop_duplicates(subset='start station id', keep='first', inplace=False, ignore_index=True)
#clean up DataFrame headers (start vs. end not relevant in this context)
station2_df = stations_df.rename(columns={'start station id':'station id','start station name':'station name','start station latitude':'station latitude','start station longitude':'station longitude'})

#view DataFrame
station2_df

,station id,station name,station latitude,station longitude
0,2009,Catherine St & Monroe St,40.711174,-73.996826
1,536,1 Ave & E 30 St,40.741444,-73.975361
2,228,E 48 St & 3 Ave,40.754601,-73.971879
3,519,Pershing Square N,40.751884,-73.977702
4,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
...,...,...,...,...
327,468,Broadway & W 55 St,40.765265,-73.981923
328,419,Carlton Ave & Park Ave,40.695807,-73.973556
329,491,E 24 St & Park Ave S,40.740964,-73.986022
330,321,Cadman Plaza E & Red Cross Pl,40.699918,-73.989718


In [5]:
#Add zip code to station DataFrame

#initialize dictionary for results of this function
results_dict = {"Zip Code":[]}

#loop through station DataFrame and collect zip codes in results_dict
for i in np.arange(0,len(station2_df)):
    result = search.by_coordinates(station2_df['station latitude'].iloc[i], station2_df['station longitude'].iloc[i])
    results_dict['Zip Code'].append(result[0].zipcode)

#construct results DataFrame
results_df = pd.DataFrame(results_dict)

#Join results to station DataFrame
station3_df = station2_df.join(results_df, how="left")

#view cleaned DataFrame
station3_df

,station id,station name,station latitude,station longitude,Zip Code
0,2009,Catherine St & Monroe St,40.711174,-73.996826,10038
1,536,1 Ave & E 30 St,40.741444,-73.975361,10016
2,228,E 48 St & 3 Ave,40.754601,-73.971879,10171
3,519,Pershing Square N,40.751884,-73.977702,10110
4,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,11217
...,...,...,...,...,...
327,468,Broadway & W 55 St,40.765265,-73.981923,10103
328,419,Carlton Ave & Park Ave,40.695807,-73.973556,11205
329,491,E 24 St & Park Ave S,40.740964,-73.986022,10001
330,321,Cadman Plaza E & Red Cross Pl,40.699918,-73.989718,11201


In [8]:
data_col2_df.to_csv(r'../citibike_2014.csv', index = False)
station3_df.to_csv(r'../stations_2014.csv', index = False)